## 1. Cargar los datos

#### 1.1. Importar librerias y módulos

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os
from warnings import simplefilter
from importlib.machinery import SourceFileLoader
from sklearn.feature_selection import VarianceThreshold
import xlsxwriter

In [2]:
fun = SourceFileLoader( 'funciones', r'..\..\..\code\modules\funciones.py' ).load_module()
vn  = SourceFileLoader( 'variables_nombres', r'..\..\..\code\modules\variables_nombres.py' ).load_module()

In [3]:
simplefilter( action = "ignore", category = pd.errors.PerformanceWarning )
simplefilter( action = "ignore", category = FutureWarning )
# print( dir( fun ) )
# pd.set_option( 'display.max_rows', 10 )
# pd.set_option( 'display.max_columns', 10 )

#### 1.2. Importar bases 

Se importan las cuatro bases de manera conjunta, y se agrupan en la lista "bases". Para accerder a cada una de las bases se llama a la lista de bases + su respectivo índice. Ejemplo: bases[0]

In [4]:
pwd

'C:\\Users\\dell\\Documents\\GitHub\\Corruption_Paper\\code\\data\\preprocess_data'

In [5]:
path = r'..\..\..\input\built_data\matrix_c1_siaf_canon_1620.dta'

In [10]:
%%time

df = pd.read_stata( path )

CPU times: total: 26.6 s
Wall time: 27 s


#### 1.3. Última limpieza a las bases

Últimas modificaciones a las bases de datos

In [ ]:
for col in df.columns:
    if "Sí" in df[ col ].values:
        print( col )

In [11]:
df[ 'corrup_intensa' ].value_counts()

corrup_intensa
Sí    989
No    389
Name: count, dtype: int64

In [12]:
data_cols        = [ col for col in df.columns if col not in vn.no_predictoras_variables and col not in vn.string_variables ]
df               = df[ data_cols ]
encode_variables = { "corrup_intensa": { "Sí": 1, "No": 0 },  "corrup_amplia": { "Sí": 1, "No": 0 } }
df               = df.replace( encode_variables )
df               = df.astype( str ).replace( '', np.nan, regex = True )
df               = df.astype( str ).replace( '\.+$', np.nan, regex = True ).astype( float )

In [13]:
df.shape

(1378, 17557)

## 2. Preprocesamiento

### 2.1 Imputación con 0

Se imputa los valores perdidos de las variables de SIAF con 0. No se generan variables dummy de control.

In [14]:
df = fun.imputar_i( df, vn.siaf_variables, 0, dummy = False )

Se verifica que las variables pertenecientes a SIAF no tengan valores perdidos

In [15]:
df.isnull().sum().sum()

0

### 2.2. Filtro por variabilidad

Se genera una lista de variables que no deben ser modificadas o filtradas

In [16]:
vars_no_mod      = [ 'ubigeo', 'year', 'variable_gasto' ]
vars_no_mod_full = vn.dependientes_variables + vars_no_mod

Se descartan aquellas variables constantes, es decir, con una variabilidad de 0

In [17]:
df = fun.filtro_variabilidad( df, 0, vars_no_mod_full )

In [18]:
df.shape

(1378, 14325)

### 2.3. Imputar outliers

Se imputa los valores del percentil 1% superior con los valores del percentil 99% para cada una de las variables de SIAF

In [19]:
df = fun.imputar_outliers( df, vn.siaf_variables, 0.01 )

### 2.7. Transformaciones logarítmicas

Se identifica las variables negativas de SIAF y se les divide entre un millón. 

In [20]:
df = fun.dividir_variables_negativas( df, 1_000_000, vars_no_mod_full )

Se crea una lista de variables a modificar logarítmicamente

In [21]:
num_predictores = vn.siaf_variables + vn.dependientes_numericas

Se realiza una transformación logarítmica a todas las variables pertenecientes a SIAF que no son negativas.

In [22]:
df = fun.transformacion_log( df, num_predictores, vars_no_mod )

Se verifica el numero de variables por fuente de cada base luego del paso 2.7.

In [23]:
df.shape

(1378, 14325)

### 2.7. Exportar las bases de datos

In [24]:
df.to_csv( r'..\..\..\input\preprocessed_data\base0_siaf_canon_1620.csv', index = False )

In [25]:
df

,year,ubigeo,piagft_reod,pimgft_reod,tejgft_reod,tdvgft_reod,dfgpimpiaft_reod,dfgdevpiagft_reod,devppimft_reod,piagft_redr,...,_pim_canonpc,_pia_canonpc,_totaleje_canonpc,_totaldev_canonpc,corrup_intensa,corrup_amplia,per_corrup1,per_corrup2,monto_corrup1,monto_corrup2
0,2016.0,10101.0,1042514.0,16.955749,22486872.37,16.751669,22066868.0,17.800800,4.413279,4112775.0,...,3.852454,0.026929,14.234422,14.234422,1.0,1.0,4.0,6.0,973126.80,973126.80
1,2018.0,10101.0,1022578.0,14.176742,0.00,14.172675,412516.0,0.406691,4.611094,3277303.0,...,4.043831,0.028230,38.567413,38.567413,1.0,1.0,4.0,0.0,850498.70,850498.70
2,2019.0,10101.0,2117193.0,15.475330,0.00,14.632606,3141169.0,0.146721,3.785407,3567725.0,...,3.644506,0.027500,0.000000,28.804354,1.0,1.0,10.0,0.0,213328.66,213328.66
3,2020.0,10101.0,2125970.0,16.550231,0.00,15.510281,13279469.0,3.319424,3.593118,3874238.0,...,3.122964,0.026807,0.000000,17.171967,1.0,1.0,5.0,0.0,25466196.00,25466196.00
4,2017.0,10102.0,4254.0,15.837134,4827767.87,15.389895,7546315.0,4.823514,4.173450,1500.0,...,5.758890,0.000000,205.684900,205.684900,1.0,1.0,5.0,0.0,7662770.00,7662770.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,2016.0,250105.0,1402637.0,15.631399,5108248.00,15.446367,4743897.0,3.705611,4.432099,2712287.0,...,4.473697,56.676807,73.471180,73.471180,1.0,1.0,16.0,0.0,19161298.00,19161298.00
1374,2017.0,250105.0,1402637.0,15.856344,7691763.84,15.842849,6294380.0,6.191212,4.601761,2828859.0,...,4.458875,37.567630,55.142548,55.142548,0.0,1.0,0.0,37.0,0.00,938897.20
1375,2018.0,250105.0,1402637.0,14.153865,0.00,14.148727,0.0,-0.007189,4.610033,2693324.0,...,4.233397,36.648098,46.291496,44.413340,0.0,1.0,0.0,2.0,0.00,762652.00
1376,2017.0,250301.0,1337089.0,16.271724,11252813.34,16.228544,10323482.0,9.830694,4.572377,1546253.0,...,5.940664,238.385640,314.466900,314.466900,0.0,1.0,0.0,5.0,0.00,1791342.00
